In [ ]:
# import json
# import matplotlib.pyplot as plt
# import numpy as np
# from datetime import datetime, timedelta, timezone
# from time import sleep

# from Functions import total_function_multiprocessing, download_show_output, getTimeseries
# from Functions import total_function_multiprocessing_lessRAM
# import pandas as pd
# pd.set_option('display.max_rows', None)

In [ ]:
# TOTAL WRAP ANALYSIS EVENTS MULTIPROCESSING 
from Functions import total_function_multiprocessing

list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_3h', 'buy_vol_6h', 'buy_vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_3h', 'vol_6h', 'vol_24h']
list_minutes = [15, 30, 60, 180, 360, 1440, 1440*3]
list_event_buy_volume = [0.65, 0.75, 0.85]
list_event_volume = [4,6,8,12,20]
n_processes = 8

LOAD_DATA = False
FREE_LOCK = True

if FREE_LOCK:
    shared_data = total_function_multiprocessing(list_buy_vol, list_vol, list_minutes, list_event_buy_volume, list_event_volume, n_processes, LOAD_DATA)



In [ ]:
# # TOTAL WRAP ANALYSIS EVENTS MULTIPROCESSING LESS RAM

# from Functions import total_function_multiprocessing_lessRAM
# list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_3h', 'buy_vol_6h', 'buy_vol_24h']
# list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_3h', 'vol_6h', 'vol_24h']
# list_minutes = [15, 30, 60, 180, 360, 1440, 1440*3]
# list_event_buy_volume = [0.65, 0.75, 0.85]
# list_event_volume = [4,6,8,12,20]
# n_processes = 8

# SLICE_I = 1
# LOAD_DATA = False
# FREE_LOCK = True

# if FREE_LOCK:
#     shared_data = total_function_multiprocessing_lessRAM(list_buy_vol, list_vol, list_minutes, list_event_buy_volume, list_event_volume, n_processes, LOAD_DATA, SLICE_I)

# import os
# duration = 2  # seconds
# freq = 440  # Hz
# os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))


In [1]:
# DOWNLOAD AND SHOW OUTPUT FROM TOTAL MULTIPROCESSING
from Functions import download_show_output
import pandas as pd
pd.set_option('display.max_rows', None)

minimum_event_number = 10
minimum_coin_number = 8
mean_threshold = 3
lb_threshold = -15
frequency_threshold = 5
group_coins = False
best_coins_volatility = 3

output, info = download_show_output(minimum_event_number=minimum_event_number, minimum_coin_number=minimum_coin_number,
                                      mean_threshold=mean_threshold, lb_threshold=lb_threshold, frequency_threshold=frequency_threshold,
                                       group_coins=group_coins, best_coins_volatility=best_coins_volatility)
df = pd.DataFrame(output).transpose()
df.sort_values(by=['mean'], ascending=False)
#df.sort_values(by=['lower_bound'], ascending=False)

/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Download completed in 6.11 seconds


,mean,std,upper_bound,lower_bound,n_coins,n_events,frequency/month
buy_vol_15m:0.75/vol_30m:20/timeframe:360/vlty:2,9.01,10.66,19.67,-1.65,15.0,18.0,11.25
buy_vol_3h:0.65/vol_6h:6/timeframe:360/vlty:2,8.99,10.92,19.91,-1.93,13.0,14.0,8.94
buy_vol_3h:0.65/vol_6h:6/timeframe:180/vlty:2,8.61,10.33,18.94,-1.72,13.0,15.0,9.57
buy_vol_15m:0.65/vol_6h:12/timeframe:4320/vlty:1,8.34,16.34,24.68,-8.00,10.0,12.0,6.10
buy_vol_15m:0.65/vol_6h:12/timeframe:1440/vlty:1,8.01,18.01,26.02,-10.00,11.0,14.0,7.12
buy_vol_5m:0.65/vol_60m:20/timeframe:4320/vlty:1,7.86,15.36,23.22,-7.50,12.0,17.0,6.80
buy_vol_15m:0.85/vol_30m:20/timeframe:1440/vlty:3,6.01,8.80,14.81,-2.79,10.0,10.0,6.38
buy_vol_24h:0.65/vol_6h:4/timeframe:1440/vlty:4,4.96,11.06,16.02,-6.10,12.0,13.0,8.30
buy_vol_24h:0.65/vol_60m:20/timeframe:1440/vlty:4,4.67,11.30,15.97,-6.63,11.0,12.0,7.66
buy_vol_24h:0.65/vol_3h:8/timeframe:1440/vlty:4,4.50,11.30,15.80,-6.80,11.0,12.0,7.66


In [ ]:
from Functions import check_invevestment_amount
check_invevestment_amount(info, output, investment_amount=90)

In [ ]:
key = 'buy_vol_5m:0.65/vol_60m:20/timeframe:4320/vlty:1'

In [ ]:
from Functions import getTimeseries
response = getTimeseries(info, key, check_past=360, look_for_newdata=True, plot=False)

In [ ]:
from Functions import infoTimeseries
#key = 'buy_vol_5m:0.65/vol_24h:8/timeframe:4320/vlty:1'
timeseries_info = infoTimeseries(info, key)
timeseries_info


In [ ]:
pd.set_option('display.max_colwidth', 1000)
#df1

In [ ]:
from Functions import RiskManagement
import pandas as pd
pd.set_option('display.max_rows', None)
#key = 'buy_vol_5m:0.65/vol_30m:20/timeframe:4320/vlty:1'
df1, df2, best_risk_key = RiskManagement(key, investment_per_event=100)
df1 = df1.sort_values(by=['mean'], ascending=False)
df2 = df2.sort_values(by=['events'], ascending=True)

#result = pd.concat([df1, df2], axis=0)
print(df1)
df2

In [2]:
from Functions import RiskConfiguration
riskmanagement_conf = (minimum_event_number, minimum_coin_number, mean_threshold, lb_threshold, frequency_threshold, group_coins, best_coins_volatility)
df  = RiskConfiguration(info, riskmanagement_conf)
df

12 keys will be analyzed in terms of risk configuration
ITERATION 1 has started
File exists, Download from local...
Status Code is :  200
All data have been downloaded
0 new events for buy_vol_5m:0.65/vol_60m:20/timeframe:4320/vlty:1
Number of combinations 486


/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1

254.21 euro of profit for an investment of 100 euro per event (total of 15 events). risk_golden_zone:1_step:0.15_step_no_golden:0.03_extratime:0.17 with 16.95 %
ITERATION 2 has started
File exists, Download from local...
Status Code is :  200
All data have been downloaded
0 new events for buy_vol_15m:0.65/vol_6h:12/timeframe:1440/vlty:1
Number of combinations 486


/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1

229.59 euro of profit for an investment of 100 euro per event (total of 13 events). risk_golden_zone:0.25_step:0.2_step_no_golden:0.05_extratime:0.33 with 17.66 %
ITERATION 3 has started
File exists, Download from local...
Status Code is :  200
All data have been downloaded
0 new events for buy_vol_15m:0.65/vol_6h:12/timeframe:4320/vlty:1
Number of combinations 486


...

/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1

140.15 euro of profit for an investment of 100 euro per event (total of 10 events). risk_golden_zone:0.8_step:0.05_step_no_golden:0.03_extratime:0.33 with 14.02 %
ITERATION 12 has started
File exists, Download from local...
Status Code is :  200
All data have been downloaded
0 new events for buy_vol_60m:0.75/vol_60m:12/timeframe:1440/vlty:3
Number of combinations 486


/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertorainieri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1

152.25 euro of profit for an investment of 100 euro per event (total of 16 events). risk_golden_zone:0.8_step:0.075_step_no_golden:0.03_extratime:0.17 with 9.52 %


,keys,golden_zone,step_golden,step_nogolden,extra_timeframe,estimated_gain,estimated_std,frequency
1,buy_vol_15m:0.65/vol_6h:12/timeframe:1440/vlty:1,0.25,0.2,0.05,0.33,17.66,49.71,7.12
0,buy_vol_5m:0.65/vol_60m:20/timeframe:4320/vlty:1,1,0.15,0.03,0.17,16.95,54.34,6.80
5,buy_vol_15m:0.75/vol_30m:20/timeframe:360/vlty:2,1,0.075,0.05,0.33,14.74,35.82,11.25
4,buy_vol_3h:0.65/vol_6h:6/timeframe:360/vlty:2,1,0.075,0.03,0.33,14.69,36.87,8.94
10,buy_vol_15m:0.85/vol_30m:20/timeframe:1440/vlty:3,0.8,0.05,0.03,0.33,14.02,30.47,6.38
3,buy_vol_3h:0.65/vol_6h:6/timeframe:180/vlty:2,1,0.075,0.01,0.17,12.43,34.92,9.57
2,buy_vol_15m:0.65/vol_6h:12/timeframe:4320/vlty:1,0.8,0.2,0.03,0.33,12.42,43.04,6.10
11,buy_vol_60m:0.75/vol_60m:12/timeframe:1440/vlty:3,0.8,0.075,0.03,0.17,9.52,26.76,14.55
9,buy_vol_15m:0.85/vol_30m:20/timeframe:180/vlty:3,0.8,0.05,0.03,0.33,8.47,24.14,6.38
8,buy_vol_24h:0.65/vol_6h:4/timeframe:1440/vlty:4,1,0.05,0.03,0.11,3.19,25.77,8.30


In [3]:
from Functions import load_riskconfiguration
df = load_riskconfiguration()
df

Estimated gain each month: 1076 euro with investment amount 100 euro per event
{
    "minimum_event_number": 10,
    "minimum_coin_number": 8,
    "mean_threshold": 3,
    "lb_threshold": -15,
    "frequency_threshold": 5,
    "group_coins": false,
    "best_coins_volatility": 3,
    "timestamp": "2023-08-02T15:37:03.925297"
}


,keys,golden_zone,step_golden,step_nogolden,extra_timeframe,estimated_gain,estimated_std,frequency
1,buy_vol_15m:0.65/vol_6h:12/timeframe:1440/vlty:1,0.25,0.2,0.05,0.33,17.66,49.71,7.12
0,buy_vol_5m:0.65/vol_60m:20/timeframe:4320/vlty:1,1,0.15,0.03,0.17,16.95,54.34,6.80
5,buy_vol_15m:0.75/vol_30m:20/timeframe:360/vlty:2,1,0.075,0.05,0.33,14.74,35.82,11.25
4,buy_vol_3h:0.65/vol_6h:6/timeframe:360/vlty:2,1,0.075,0.03,0.33,14.69,36.87,8.94
10,buy_vol_15m:0.85/vol_30m:20/timeframe:1440/vlty:3,0.8,0.05,0.03,0.33,14.02,30.47,6.38
3,buy_vol_3h:0.65/vol_6h:6/timeframe:180/vlty:2,1,0.075,0.01,0.17,12.43,34.92,9.57
2,buy_vol_15m:0.65/vol_6h:12/timeframe:4320/vlty:1,0.8,0.2,0.03,0.33,12.42,43.04,6.10
11,buy_vol_60m:0.75/vol_60m:12/timeframe:1440/vlty:3,0.8,0.075,0.03,0.17,9.52,26.76,14.55
9,buy_vol_15m:0.85/vol_30m:20/timeframe:180/vlty:3,0.8,0.05,0.03,0.33,8.47,24.14,6.38
8,buy_vol_24h:0.65/vol_6h:4/timeframe:1440/vlty:4,1,0.05,0.03,0.11,3.19,25.77,8.30


In [6]:
import shutil
import os
from random import randint
from datetime import datetime
ROOT_PATH = os.getcwd()
random_id = str(randint(1,1000))
now = datetime.now()
day = str(now.day)
month = str(now.month)
year = str(now.year)

src = ROOT_PATH + "/riskmanagement_json/riskmanagement.json"
dst = f"{ROOT_PATH}/riskmanagement_backup/riskmanagement-{day}-{month}-{year}-{random_id}.json"
shutil.copyfile(src, dst)

'/Users/albertorainieri/Projects/Personal/analysis/riskmanagement_backup/riskmanagement-2-8-2023-35.json'